In [1]:
import cv2
import numpy as np
import math

In [2]:
def rgb2gray(rgb):
    return np.round(np.dot(rgb[...,:3], [0.299, 0.587, 0.114]))

# def rgb2gray(rgb):

#     r, g, b = rgb[:,:,0], rgb[:,:,1], rgb[:,:,2]
#     gray = 0.299 * r + 0.587 * g + 0.114 * b

#     return gray

img = cv2.imread('C:/Users/anvit/OneDrive/Desktop/HoG/Training images (Pos)/crop_000010b.bmp')  
img_array = np.array(img, dtype=float)
gray = rgb2gray(img)
img_gray = cv2.imwrite('C:/Users/anvit/OneDrive/Desktop/HoG/Training images (Pos)/crop_000010b_gray.bmp', gray)
gray_array = np.array(gray, dtype=float)

In [3]:
print(gray)

[[ 64.  64.  89. ... 178. 182. 154.]
 [ 64.  64.  89. ... 178. 182. 154.]
 [ 70.  70.  88. ... 179. 151. 122.]
 ...
 [124. 124. 117. ...  55.  56.  55.]
 [124. 124. 117. ...  55.  56.  55.]
 [124. 124. 117. ...  55.  56.  55.]]


In [4]:
## Sobels Operator

#sobels operator
def sobels_operator(inp):
    g_x = np.array([[-1,0,1],
                   [-2,0,2],
                   [-1,0,1]])
    g_y = np.array([[1,2,1],
                   [0,0,0],
                   [-1,-2,-1]])
    
    #Horizontal Gradient 
    G_X = conv_sobel(inp,g_x)
    #Saving the image
    cv2.imwrite('C:/Users/anvit/OneDrive/Desktop/HoG/Training images (Pos)/crop_000010b_gray_sobelGX.bmp', G_X)
       
    #Vertical Gradient
    G_Y = conv_sobel(inp,g_y)
    #saving the image
    cv2.imwrite('C:/Users/anvit/OneDrive/Desktop/HoG/Training images (Pos)/crop_000010b_gray_sobelGY.bmp', G_Y)
    
    #Gradient Magnitude
    g_mag = np.sqrt(np.square(G_X) + np.square(G_Y))
    #saving the image
    cv2.imwrite('C:/Users/anvit/OneDrive/Desktop/HoG/Training images (Pos)/crop_000010b_gray_sobel.bmp', g_mag)    
    return G_X,G_Y, g_mag

#convolution done by implementing using 2D directly
def conv_sobel(x,y):
    
    x_copy = x.copy()
    #height and width of the image
    xH = x.shape[0]
    xW = x.shape[1]
    #a new array to store the output of convolution
    sobel_image = np.zeros((xH, xW))
    
    #height and width of the mask
    yH = y.shape[0]
    yW = y.shape[1]  
    
    #for loopinng inside the boundary, we divide by 2
    yH_new = int((yH) // 2)
    yW_new = int((yW) // 2)
    
    k = int(np.floor(y.shape[0]/2))
#     print(k)
    #selecting the range so the we only consider pixels that are inside the boundary/border
    for i in range(k,xH-k):
        for j in range(k,xW-k):
            
            #consider a variable to calculate the summation over the mask
            summation = 0
#             x_copy[i][j] = np.sum(x[i-k:i+k+1,j-k:j+k+1] * y)
#     final_img = x_copy[k:x.shape[0]-k, k:x.shape[1]-k]/4
            #to check the corresponding mask value , we have to set ranges
            for k in np.arange(-yH_new, yH_new+1):
                for l in np.arange(-yW_new, yW_new+1):
                    
                    #m is the value of the corresponding image pixel
                    m = x[i+k, j+l]
                    #n is the value of the corresponding mask pixel
                    n = y[yH_new+k, yW_new+l]
                    #summation is the sum of their products over the range
                    summation += (m * n)
            
            #To normalize our image to contain values from 0 to 255 we divide the summation by 4
            sobel_image[i,j] = summation / 4
            
    return sobel_image


In [5]:
g_x,g_y,sobel_output = np.round(sobels_operator(gray))

In [6]:
print(sobel_output[28][28])

25.0


In [7]:
#Finding the Gradient Angles i.e., theta
#arctan2 is tan inverse, which gives us results in radians with [-pi to pi]
theta = np.zeros((gray_array.shape[0],gray_array.shape[1]))
for i in range(0, gray_array.shape[0]):
        for j in range(0, gray_array.shape[1]):
            if(g_y[i][j] == 0 and g_x[i][j] == 0):
                theta[i][j] = 0
            else:
                theta[i][j] = np.arctan2(g_y[i][j],g_x[i][j])
    
#Converting radians to degrees
theta = np.rad2deg(theta)
#converting all the negatives into positives by adding 360 so the range is now [0 to 360]
for i in range(0, gray_array.shape[0]):
        for j in range(0, gray_array.shape[1]):
            if(theta[i][j] < 0):
                theta[i][j] += 360

In [8]:
print(g_x[28][28])
print(g_y[28][28])
print(theta[28][28])
# angle = theta
# for i in range(0, gray_array.shape[0]):
#         for j in range(0, gray_array.shape[1]):
#             if(angle[i][j] >= 170 and angle[i][j] < 350):
#                 angle[i][j] -= 180
#             if(angle[i][j]<0 and angle[i][j] >= -20):
#                 print(angle[i][j])
# #             if(angle[i][j] == -20):
                
# print(angle)
print(theta[28][28])

8.0
-24.0
288.434948822922
288.434948822922


In [9]:
# histogram_centers = np.array([0,20,40,60,80,100,120,140,160])
# num_bins = 9
# # bins = [None] * 1


In [10]:
class Histogram:
    def __init__(self, mag_block, theta_block):
        self.center = np.array([0,20,40,60,80,100,120,140,160])
#         4 histograms of 9 bins each - per block
        self.bins = np.zeros((4,9), dtype=float)
        self.cell_size = 8
        self.cells = []
        self.t_cells = []
        self.m_cells = []
        self.theta_block = theta_block
        self.mag_block = mag_block
        self.convert_block_2_cell(mag_block,theta_block)
#         c=0
#         print(c)
        self.generate_features(self.m_cells, self.t_cells)
        self.norm_hog()
    
    def norm_hog(self):
        
        summation = 0
        for bin in self.bins:
            for j in range(len(bin)):
                summation += bin[j] ** 2
        
        dist = summation ** 0.5
                   
        for bin in self.bins:
            for j in range(len(bin)):
                
                bin[j] /= dist

    
    def convert_block_2_cell(self, mag_block, theta_block):
        c=0
        for m in range(0,theta_block.shape[0], self.cell_size):
                for n in range(0, theta_block.shape[1], self.cell_size):
                    
                    m_cell = np.zeros((self.cell_size,self.cell_size))
                    t_cell = np.zeros((self.cell_size,self.cell_size))
                    
                    for p in range(self.cell_size):
                        for q in range(self.cell_size):
                            
                            m_cell[p][q] = mag_block[m+p][n+q]
                            t_cell[p][q] = theta_block[m+p][n+q]
                     
                    c+=1
                    self.m_cells.append(m_cell)
                    self.t_cells.append(t_cell)
#         print(c)
    
    def generate_features(self, m_cells, t_cells):
        
        for i in range(len(t_cells)):
            for j in range(len(t_cells[i])):
                for k in range(len(t_cells[i][j])):
                    self.cal_hist(m_cells[i][j][k],t_cells[i][j][k], self.bins[i]) 
                
# HANDLE EDGE CASE : -20,10       
    def cal_hist(self, mag, angle, bins):       
        
#         COMPUTE THE DISTANCE TO ADD THE MAG WRT THE CENTER
        if(angle >= 180):
            angle -= 180
#         now we have all unsigned angles
# new case to consider: if angle is >=160 first = bin 9 and second is bin 1
        
        if(angle >= 160):
            first_bin = 8
            second_bin = 0
            percentage = self.calc_distance(angle, self.center[first_bin])
            bins[first_bin] += (1-percentage) * mag
            bins[second_bin] += percentage * mag 
#         edge care , if angle is >=350 it's between bins 1 and 9
        
        if(angle >= 0 and angle < 20):
            first_bin = 0
            second_bin = 1
            percentage = self.calc_distance(angle, self.center[first_bin])
            bins[first_bin] += (1-percentage) * mag
            bins[second_bin] += percentage * mag
        
        if(angle >= 20 and angle < 40):
            first_bin = 1
            second_bin = 2
            percentage = self.calc_distance(angle, self.center[first_bin])
            bins[first_bin] += (1-percentage) * mag
            bins[second_bin] += percentage * mag
            
        if(angle >= 40 and angle < 60):
            first_bin = 2
            second_bin = 3
            percentage = self.calc_distance(angle, self.center[first_bin])
            bins[first_bin] += (1-percentage) * mag
            bins[second_bin] += percentage * mag
            
        if(angle >= 60 and angle < 80):
            first_bin = 3
            second_bin = 4
            percentage = self.calc_distance(angle, self.center[first_bin])
            bins[first_bin] += (1-percentage) * mag
            bins[second_bin] += percentage * mag
        
        if(angle >= 80 and angle < 100):
            first_bin = 4
            second_bin = 5
            percentage = self.calc_distance(angle, self.center[first_bin])
            bins[first_bin] += (1-percentage) * mag
            bins[second_bin] += percentage * mag
        
        if(angle >= 100 and angle < 120):
            first_bin = 5
            second_bin = 6
            percentage = self.calc_distance(angle, self.center[first_bin])
            bins[first_bin] += (1-percentage) * mag
            bins[second_bin] += percentage * mag
        
        if(angle >= 120 and angle < 140):
            first_bin = 6
            second_bin = 7
            percentage = self.calc_distance(angle, self.center[first_bin])
            bins[first_bin] += (1-percentage) * mag
            bins[second_bin] += percentage * mag
         
        if(angle >= 140 and angle < 160):
            first_bin = 7
            second_bin = 8
            percentage = self.calc_distance(angle, self.center[first_bin])
            bins[first_bin] += (1-percentage) * mag
            bins[second_bin] += percentage * mag
            
    def calc_distance(self, angle, center):
        percent = (np.absolute(angle - center))/20
        return percent

In [11]:
def h_feature(mag, theta): 
    descriptor = []
    cell_size = 8
    block_size = 16
    block_overlap = 8
    assert(mag.shape[0] == theta.shape[0])
    assert(mag.shape[1] == theta.shape[1])
    b=0
    c=0
    
#     generating blocks from pixels
    for i in range(0,theta.shape[0]-cell_size, cell_size):
        for j in range(0,theta.shape[1]-cell_size, cell_size):
            
            m_block = np.zeros((block_size,block_size))
            t_block = np.zeros((block_size,block_size))
            
            for k in range(block_size):
                for l in range(block_size):  
                    
                    m_block[k][l] = mag[i+k][j+l]
                    t_block[k][l] = theta[i+k][j+l]
            b +=1
                    
#             m_cell = np.zeros((cell_size,cell_size))
#             t_cell = np.zeros((cell_size,cell_size))
            
#             for m in range(0,t_block.shape[0], cell_size):
#                 for n in range(0, t_block.shape[1], cell_size):
                    
#                     for p in range(cell_size):
#                         for q in range(cell_size):
                            
#                             m_cell[p][q] = m_block[m+p][n+q]
#                             t_cell[p][q] = t_block[m+p][n+q]
                            
#                     c +=1
            hist_obj = Histogram(m_block,t_block)
            descriptor.append(hist_obj.bins)
    print(b)
#     print(c)
    return descriptor


In [12]:
h_feature_output = h_feature(sobel_output,theta)
print(h_feature_output)

209
[array([[0.05316949, 0.00646588, 0.00448384, 0.02005229, 0.25231975,
        0.42141096, 0.16570556, 0.12344027, 0.12716887],
       [0.12435825, 0.07434849, 0.05849574, 0.06433934, 0.20835146,
        0.12724127, 0.08100616, 0.09487015, 0.16220603],
       [0.07262254, 0.0384686 , 0.05664406, 0.04603198, 0.21248879,
        0.21220456, 0.34597415, 0.23618491, 0.20964266],
       [0.07581757, 0.04570324, 0.04749158, 0.06792334, 0.20082099,
        0.24937184, 0.273385  , 0.22587089, 0.04344412]]), array([[0.14265122, 0.08528509, 0.06710041, 0.0738036 , 0.23899976,
        0.14595833, 0.09292209, 0.10882546, 0.18606638],
       [0.10001267, 0.173583  , 0.16204968, 0.12783285, 0.06967855,
        0.11450302, 0.26236517, 0.2406658 , 0.10023934],
       [0.08697026, 0.05242614, 0.05447755, 0.0779148 , 0.23036156,
        0.2860542 , 0.31359966, 0.25909628, 0.04983471],
       [0.13829284, 0.12696628, 0.06907765, 0.05526745, 0.12921667,
        0.19293315, 0.17343583, 0.33930209, 0.1643

In [13]:
class LBP:
    def __init__(self, gray_block):
        self.mag = gray_block
        self.bins = np.zeros((59,), dtype=float)
        self.generate_features(gray_block)
        self.norm_lbp()
        
    def norm_lbp(self):
        for i in range(len(self.bins)):
            self.bins[i] = self.bins[i] / 256
    
    def generate_features(self,gray_block):
        for i in range(gray_block.shape[0]):
            for j in range(gray_block.shape[1]):
                self.compute_LBP_pattern(i,j,gray_block)
        
#     def norm_lbp(self):
#         for i in range(len(self.bins)):
#             self.bins[i] = self.bins[i] / 256
        
    def compute_LBP_pattern(self,i, j,gray_block):
        x = gray_block
        pattern = ''
        
        try:
            if(x[i-1][j-1] > x[i][j]):
                pattern += '1'
            else:
                pattern += '0'
                
            if(x[i-1][j] > x[i][j]):
                pattern += '1'
            else:
                pattern += '0'
                
            if(x[i-1][j+1] > x[i][j]):
                pattern += '1'
            else:
                pattern += '0'
                
            if(x[i][j+1] > x[i][j]):
                pattern += '1'
            else:
                pattern += '0'
                
            if(x[i+1][j+1] > x[i][j]):
                pattern += '1'
            else:
                pattern += '0'
            
            if(x[i+1][j] > x[i][j]):
                pattern += '1'
            else:
                pattern += '0'
            
            if(x[i+1][j-1] > x[i][j]):
                pattern += '1'
            else:
                pattern += '0'
            
            if(x[i][j-1] > x[i][j]):
                pattern += '1'
            else:
                pattern += '0'
                
            self.append_to_bins(pattern)
            
        except IndexError:
            pattern = '00000101'
            self.append_to_bins(pattern)
        
        
        
    def append_to_bins(self, lbp_pattern):        
        decimal = int(lbp_pattern, 2)
        patterns = {
            0: 0,
            1: 1,
            2: 2,
            3: 3,
            4: 4,
            6: 5,
            7: 6,
            8: 7,
            12: 8,
            14: 9,
            15: 10,
            16: 11,
            24: 12,
            28: 13,
            30: 14,
            31: 15,
            32: 16,
            48: 17,
            56: 18,
            60: 19,
            62: 20,
            63: 21,
            64: 22,
            96: 23,
            112: 24,
            120: 25,
            124: 26,
            126: 27,
            127: 28,
            128: 29,
            129: 30,
            131: 31,
            135: 32,
            143: 33,
            159: 34,
            191: 35,
            192: 36,
            193: 37,
            195: 38,
            199: 39,
            207: 40,
            223: 41,
            224: 42,
            225: 43,
            227: 44,
            231: 45,
            239: 46,
            240: 47,
            241: 48,
            243: 49,
            247: 50,
            248: 51,
            249: 52,
            251: 53,
            252: 54,
            253: 55,
            254: 56,
            255: 57
        }
        bin_number = patterns.get(decimal, 58)
        self.bins[bin_number] += 1   
        
    

In [14]:
def LBP_feature(gray):
    lbp_descriptor = []
    cell_size = 8
    block_size = 16
    block_overlap = 8
    b=0
#     c=0
    
#     generating blocks from pixels   
    for i in range(0,gray.shape[0],block_size):
        for j in range(0,gray.shape[1],block_size):
            
            gray_block = np.zeros((block_size,block_size))
            
            for p in range(block_size):
                for q in range(block_size):
                    
                    gray_block[p][q] = gray[i+p][j+q]
            
            
            b+=1
            lbp_obj1 = LBP(gray_block)
            lbp_descriptor.append(lbp_obj1.bins)
#     print(b)        
    return lbp_descriptor
    

In [15]:
lbp_output = LBP_feature(gray)

In [16]:
print(lbp_output)

[array([0.05859375, 0.01953125, 0.01171875, 0.00390625, 0.0078125 ,
       0.00390625, 0.00390625, 0.0078125 , 0.01953125, 0.01171875,
       0.        , 0.01171875, 0.01953125, 0.00390625, 0.03125   ,
       0.0078125 , 0.0078125 , 0.0078125 , 0.0078125 , 0.02734375,
       0.046875  , 0.        , 0.00390625, 0.01953125, 0.        ,
       0.        , 0.00390625, 0.00390625, 0.00390625, 0.        ,
       0.        , 0.0078125 , 0.00390625, 0.02734375, 0.        ,
       0.015625  , 0.01171875, 0.015625  , 0.015625  , 0.00390625,
       0.00390625, 0.        , 0.03515625, 0.0390625 , 0.03125   ,
       0.00390625, 0.0234375 , 0.02734375, 0.015625  , 0.        ,
       0.00390625, 0.02734375, 0.0078125 , 0.00390625, 0.0078125 ,
       0.0078125 , 0.01171875, 0.01953125, 0.2734375 ]), array([0.06640625, 0.01171875, 0.00390625, 0.03125   , 0.0078125 ,
       0.015625  , 0.015625  , 0.00390625, 0.01171875, 0.015625  ,
       0.01171875, 0.01171875, 0.0234375 , 0.01171875, 0.01171875,
    

In [17]:
# NORMALIZE LBP BLOCKS TOO 16*16 THAT IS DIVIDE BY 256 THSI IS DIFFERENT TO HOG